### **SpaceX Falcon 9 Birinci Aşama İniş Tahmini**


### **Amaçlar (Objectives)**

* Keşifsel Veri Analizi (Exploratory Data Analysis) yap
* Eğitim Etiketlerini Belirle (Determine Training Labels)

---

### **Keşifsel Veri Analizi (Exploratory Data Analysis)**

* Veriyi inceleyerek anlamlı bilgiler çıkar
* Modeli eğitmek için kullanılacak etiketleri oluştur


---

### **Kütüphaneleri içe aktar ve yardımcı fonksiyonları tanımla**

Bu analizde kullanmak üzere aşağıdaki kütüphaneleri Python’a dahil edeceğiz.



In [1]:
# Pandas, Python programlama dili için yazılmış, veri işleme ve analiz için kullanılan bir yazılım kütüphanesidir.
import pandas as pd

# NumPy, Python için yazılmış; büyük, çok boyutlu diziler ve matrisler ile çalışmayı kolaylaştıran bir kütüphanedir.
# Ayrıca bu diziler üzerinde işlem yapmayı sağlayan birçok gelişmiş matematiksel fonksiyon içerir.
import numpy as np


### Veri Analizi

Önceki bölümden SpaceX veri setini yükle.


In [2]:
df=pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/dataset_part_1.csv")
df.head(10)

,FlightNumber,Date,BoosterVersion,PayloadMass,Orbit,LaunchSite,Outcome,Flights,GridFins,Reused,Legs,LandingPad,Block,ReusedCount,Serial,Longitude,Latitude
0,1,2010-06-04,Falcon 9,6104.959412,LEO,CCAFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B0003,-80.577366,28.561857
1,2,2012-05-22,Falcon 9,525.000000,LEO,CCAFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B0005,-80.577366,28.561857
2,3,2013-03-01,Falcon 9,677.000000,ISS,CCAFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B0007,-80.577366,28.561857
3,4,2013-09-29,Falcon 9,500.000000,PO,VAFB SLC 4E,False Ocean,1,False,False,False,NaN,1.0,0,B1003,-120.610829,34.632093
4,5,2013-12-03,Falcon 9,3170.000000,GTO,CCAFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B1004,-80.577366,28.561857
5,6,2014-01-06,Falcon 9,3325.000000,GTO,CCAFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B1005,-80.577366,28.561857
6,7,2014-04-18,Falcon 9,2296.000000,ISS,CCAFS SLC 40,True Ocean,1,False,False,True,NaN,1.0,0,B1006,-80.577366,28.561857
7,8,2014-07-14,Falcon 9,1316.000000,LEO,CCAFS SLC 40,True Ocean,1,False,False,True,NaN,1.0,0,B1007,-80.577366,28.561857
8,9,2014-08-05,Falcon 9,4535.000000,GTO,CCAFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B1008,-80.577366,28.561857
9,10,2014-09-07,Falcon 9,4428.000000,GTO,CCAFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B1011,-80.577366,28.561857


Her bir özellikteki (attribute) eksik değerleri tespit et ve yüzdesini hesapla.


In [3]:
df.isnull().sum()/len(df)*100

FlightNumber       0.000000
Date               0.000000
BoosterVersion     0.000000
PayloadMass        0.000000
Orbit              0.000000
LaunchSite         0.000000
Outcome            0.000000
Flights            0.000000
GridFins           0.000000
Reused             0.000000
Legs               0.000000
LandingPad        28.888889
Block              0.000000
ReusedCount        0.000000
Serial             0.000000
Longitude          0.000000
Latitude           0.000000
dtype: float64

Hangi sütunların sayısal (numerical) ve hangi sütunların kategorik (categorical) olduğunu belirle.


In [4]:
df.dtypes

FlightNumber        int64
Date               object
BoosterVersion     object
PayloadMass       float64
Orbit              object
LaunchSite         object
Outcome            object
Flights             int64
GridFins             bool
Reused               bool
Legs                 bool
LandingPad         object
Block             float64
ReusedCount         int64
Serial             object
Longitude         float64
Latitude          float64
dtype: object

### GÖREV 1: Her fırlatma sahasındaki fırlatma sayısını hesapla

Veri seti, birkaç farklı SpaceX fırlatma tesisini içermektedir:

* **Cape Canaveral Space Launch Complex 40**,
* **Vandenberg Air Force Base Space Launch Complex 4E (SLC-4E)**
* **Kennedy Space Center Launch Complex 39A (KSC LC 39A)**

Her fırlatmanın yeri, **LaunchSite** sütununda yer almaktadır.

---

Şimdi her bir fırlatma sahasında kaç kez fırlatma yapıldığını görelim.

**LaunchSite** sütununda `.value_counts()` metodunu kullanarak her saha için fırlatma sayısını belirle:


In [5]:
df['LaunchSite'].value_counts()

LaunchSite
CCAFS SLC 40    55
KSC LC 39A      22
VAFB SLC 4E     13
Name: count, dtype: int64

Her fırlatma, belirli bir yörüngeye ulaşmayı hedefler ve işte bazı yaygın yörünge türleri:

**LEO (Alçak Dünya Yörüngesi):** LEO, Dünya merkezli ve 2.000 km (1.200 mil) ya da daha düşük irtifaya sahip bir yörüngedir (bu, Dünya yarıçapının yaklaşık üçte biri kadardır). Ayrıca günde en az 11.25 tur (128 dakikadan kısa bir yörünge süresi) yapan ve dışmerkezliği 0.25’ten az olan yörüngeler de LEO sınıfına girer. İnsan yapımı uzay nesnelerinin çoğu LEO’da bulunur.

**VLEO (Çok Alçak Dünya Yörüngesi):** Ortalama irtifası 450 km’nin altında olan yörüngelerdir. Bu yörüngelerde çalışan Dünya gözlem uyduları, Dünya’ya daha yakın oldukları için gözlem açısından birçok avantaja sahip olabilir.

**GTO (Jeosenkron Transfer Yörüngesi):** Jeosenkron yörünge, Dünya'nın dönüşüyle senkronize olan yüksek irtifalı bir yörüngedir. Dünya ekvatorunun 35.786 kilometre (22.236 mil) yukarısında bulunur. Bu yörünge konumu, hava durumu izleme, iletişim ve gözetleme açısından çok değerlidir. Uydu, Dünya’nın dönüş hızına eşit bir hızla döndüğü için, gökyüzünde sabit bir konumda gibi görünür.

**SSO (Güneş Eşzamanlı Yörünge):** Hemen hemen kutupları kapsayan bu yörüngede uydu, Dünya üzerindeki herhangi bir noktadan her geçişinde aynı yerel güneş zamanında geçer. Bu yüzden Dünya gözlemleri için çok kullanışlıdır.

**ES-L1 (Dünya-Güneş Lagrange Noktası 1):** Lagrange noktalarında, iki büyük kütlenin (örneğin Dünya ve Güneş) yerçekimi kuvvetleri, arada kalan küçük bir cismin dengede kalabileceği şekilde birbirini dengeler. L1 noktası, Güneş ile Dünya arasında yer alır ve genellikle Güneş gözlemi için tercih edilir.

**HEO (Yüksek Eliptik Yörünge):** Dünya etrafında oldukça eliptik ve yüksek dışmerkezli yörüngelerdir. Genellikle yörüngenin bir ucunda Dünya’ya çok yaklaşırken diğer ucunda oldukça uzaklaşır.

**ISS (Uluslararası Uzay İstasyonu):** LEO’da bulunan modüler bir uzay istasyonudur. NASA (ABD), Roscosmos (Rusya), JAXA (Japonya), ESA (Avrupa) ve CSA (Kanada) gibi beş uzay ajansının işbirliğiyle inşa edilmiştir ve yaşanabilir bir yapay uydu olarak görev yapmaktadır.

**MEO (Orta Dünya Yörüngesi):** 2.000 km ile 35.786 km arasındaki irtifalarda yer alan jeosantrik yörüngelerdir. Genellikle 20.200 km veya 20.650 km yükseklikte bulunurlar ve yörünge süreleri yaklaşık 12 saattir.

**HEO (Yüksek Dünya Yörüngesi):** 35.786 km’nin üzerindeki irtifalarda bulunan jeosantrik yörüngelerdir.

**GEO (Jeostatik Yörünge):** Dünya’nın ekvatoru üzerinde 35.786 km yükseklikte, Dünya’nın dönüş yönüyle aynı yönde dönen dairesel yörüngedir. Uydu, Dünya’ya göre sabit bir noktada kalır.

**PO (Kutup Yörüngesi):** Uydu, Dünya’nın kutuplarının üzerinden ya da çok yakınından geçer. Dünya’nın tamamını taramak için uygundur.


### Görev 2: Her yörünge türünün sayısını ve tekrarını hesaplayın
Orbit sütunundaki her yörüngenin sayısını ve kaç kez geçtiğini belirlemek için `.value_counts()` metodunu kullanın.


In [7]:
# Orbit sütunundaki her yörünge türünün sayısını hesapla
df['Orbit'].value_counts()

Orbit
GTO      27
ISS      21
VLEO     14
PO        9
LEO       7
SSO       5
MEO       3
ES-L1     1
HEO       1
SO        1
GEO       1
Name: count, dtype: int64

### Görev 3: Yörüngelerin görev sonuçlarının sayısını ve tekrarını hesaplayın
`Outcome` sütununda `.value_counts()` metodunu kullanarak iniş sonuçlarının (landing outcomes) sayısını belirleyin. Sonucu `landing_outcomes` değişkenine atayın.


In [8]:
# Outcome sütunundaki her görev sonucunun sayısını hesapla ve landing_outcomes değişkenine ata
landing_outcomes = df['Outcome'].value_counts()


**True Ocean**, görevin başarıyla belirli bir okyanus bölgesine iniş yaptığını;

**False Ocean**, görevin belirli bir okyanus bölgesine başarısız iniş yaptığını;

**True RTLS**, görevin başarıyla bir kara iniş pistine iniş yaptığını;

**False RTLS**, görevin bir kara iniş pistine başarısız iniş yaptığını;

**True ASDS**, görevin başarıyla bir drone gemisine iniş yaptığını;

**False ASDS**, görevin bir drone gemisine başarısız iniş yaptığını ifade eder.

**None ASDS** ve **None None** ise inişin başarısız olduğunu gösterir.


In [9]:
# landing_outcomes içindeki anahtarları (görev sonuçlarını) indeksleriyle beraber döngüyle gez
for i, outcome in enumerate(landing_outcomes.keys()):
    # İndeks ve görev sonucunu ekrana yazdır
    print(i, outcome)

0 True ASDS
1 None None
2 True RTLS
3 False ASDS
4 True Ocean
5 False Ocean
6 None ASDS
7 False RTLS


İkinci aşamanın inişi başarılı olmayan sonuçlardan oluşan bir küme oluşturuyoruz:


In [10]:
# landing_outcomes anahtarlarını alıyoruz (görev sonuçları)
# Sonra indeksleri 1, 3, 5, 6 ve 7 olan öğeleri seçip set (küme) yapıyoruz
bad_outcomes = set(landing_outcomes.keys()[ [1, 3, 5, 6, 7] ])

# Oluşturduğumuz kötü sonuçlar kümesini yazdırıyoruz
bad_outcomes

{'False ASDS', 'False Ocean', 'False RTLS', 'None ASDS', 'None None'}

### Görev 4: Outcome sütunundan iniş sonucu etiketi oluşturun
Outcome sütununu kullanarak, eğer ilgili satırdaki değer `bad_outcomes` kümesindeyse liste elemanı sıfır, değilse bir olacak şekilde bir liste oluşturun. Daha sonra bu listeyi `landing_class` değişkenine atayın.


In [11]:
# landing_class elemanı 0 olacak eğer Outcome değeri bad_outcomes kümesindeyse
# landing_class elemanı 1 olacak aksi halde

landing_class = [0 if outcome in bad_outcomes else 1 for outcome in df['Outcome']]

print(landing_class)


[0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


Bu değişken, her fırlatmanın sonucunu temsil eden sınıflandırma değişkeni olacaktır. Değer sıfır ise birinci aşama başarılı bir şekilde iniş yapmamış demektir; bir ise birinci aşama başarılı bir şekilde iniş yapmış demektir.

In [12]:
# landing_class listesini DataFrame'e 'Class' adıyla yeni bir sütun olarak ekle
df['Class'] = landing_class

# 'Class' sütununun ilk 8 satırını görüntüle
print(df[['Class']].head(8))

   Class
0      0
1      0
2      0
3      0
4      0
5      0
6      1
7      1


In [13]:
df.head(5)

,FlightNumber,Date,BoosterVersion,PayloadMass,Orbit,LaunchSite,Outcome,Flights,GridFins,Reused,Legs,LandingPad,Block,ReusedCount,Serial,Longitude,Latitude,Class
0,1,2010-06-04,Falcon 9,6104.959412,LEO,CCAFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B0003,-80.577366,28.561857,0
1,2,2012-05-22,Falcon 9,525.000000,LEO,CCAFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B0005,-80.577366,28.561857,0
2,3,2013-03-01,Falcon 9,677.000000,ISS,CCAFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B0007,-80.577366,28.561857,0
3,4,2013-09-29,Falcon 9,500.000000,PO,VAFB SLC 4E,False Ocean,1,False,False,False,NaN,1.0,0,B1003,-120.610829,34.632093,0
4,5,2013-12-03,Falcon 9,3170.000000,GTO,CCAFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B1004,-80.577366,28.561857,0


Aşağıdaki kod satırını kullanarak başarı oranını belirleyebiliriz:


In [14]:
df["Class"].mean()

0.6666666666666666

Şimdi bunu bir sonraki bölüm için CSV dosyasına aktarabiliriz, ancak cevapların tutarlı olması için sonraki laboratuvarda verileri önceden seçilmiş bir tarih aralığında sağlayacağız.

In [15]:
df.to_csv("dataset_part_2.csv", index=False)

In [16]:
# True ASDS olan görevlerin sayısı
successful_asds = (df['Outcome'] == 'True ASDS').sum()

print(f"Bir drone gemisine başarıyla inen görev sayısı: {successful_asds}")


Bir drone gemisine başarıyla inen görev sayısı: 41
